In [ ]:
import os
import pdf2image
from PIL import Image
import pytesseract
import difflib
import re
import pandas as pd

In [ ]:
def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
DATA_PATH = '../data/'
STATE = 'mp/'

PARSE_DATA_PAGES = "../parseData/images/"+STATE
create_path(PARSE_DATA_PAGES)

PARSE_DATA_BLOCKS = "../parseData/blocks/"+STATE
create_path(PARSE_DATA_BLOCKS)

PARSE_DATA_CSVS = "../parseData/csvs/"+STATE
create_path(PARSE_DATA_CSVS)

COLUMNS = ["number","id", "elector_name", "father_or_husband_name", "relationship", "house_no", "age", "sex", "ac_name", "parl_constituency", "part_no", "year", "state", "filename", "main_town", "police_station", "mandal", "revenue_division", "district", "pin_code", "polling_station_name", "polling_station_address", "net_electors_male", "net_electors_female", "net_electors_third_gender", "net_electors_total","original_or_amendment"]

state_pdfs_path = DATA_PATH+STATE
state_pdfs_files = os.listdir(state_pdfs_path)

In [ ]:
state_pdfs_files

In [ ]:
def pdf_to_img(pdf_file_path, output_images_path,dpi=200,page=None) :
    
    PDF_PATH = pdf_file_path
    DPI = dpi
    OUTPUT_FOLDER = output_images_path
    FIRST_PAGE = page
    LAST_PAGE = page
    FORMAT = 'jpg'
    THREAD_COUNT = 1
    USERPWD = None
    USE_CROPBOX = False
    STRICT = False
    
    def rename_filename(output_image_path,idx):
        path, filename = os.path.split(output_image_path)
        os.rename(output_image_path,path+"/"+str(idx)+".jpg")
    
    def delete_existing_images():
        images = os.listdir(output_images_path)
        for image in images:
            os.remove(output_images_path+"/"+image)

    

    def pdftopil():

        pil_images = pdf2image.convert_from_path(PDF_PATH, 
                                                 dpi=DPI, 
                                                 output_folder=OUTPUT_FOLDER, 
                                                 first_page=FIRST_PAGE, 
                                                 last_page=LAST_PAGE, 
                                                 fmt=FORMAT, 
                                                 thread_count=THREAD_COUNT, 
                                                 userpw=USERPWD, 
                                                 use_cropbox=USE_CROPBOX, 
                                                 strict=STRICT)
       
        for idx,image in enumerate(pil_images,1):
            rename_filename(image.filename,idx)
            
        return pil_images
    
    
    delete_existing_images()
    pil_images = pdftopil()
    
    return pil_images


In [ ]:
def tryint(s):
    try:
        return int(s)
    except ValueError:
        return s
    
def alphanum_key(s):
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    l.sort(key=alphanum_key)

In [ ]:
def crop_section(intial_width,intial_height,crop_width,crop_height,img):
    area = (intial_width, intial_height, intial_width+crop_width, intial_height+crop_height)
    cropped_img = img.crop(area)
    return cropped_img

def split_data(data):
    data = data.split(":")
    data = [ i for i in data if i.strip()!='']
    if len(data)>1:
        data = data[1].strip()
        return data
    else:
        data = ""

In [ ]:
def generate_poll_blocks_from_page(page_full_path,page_blocks_path,amend_page):
    
    img = Image.open(page_full_path)

    amend = False
    
    def generate(intial_width,a,b,gap):
        count = 0
        crop_width = 1243
        crop_height = 490

        for col in range(1,11):

            for row in range(1,4):
                c = a+crop_width
                d = b+crop_height
                area = (a, b, c, d)
                cropped_img = img.crop(area)
                count = count+1
                cropped_img.save(page_blocks_path+str(count)+".jpg")

                a = c

            a = intial_width
            b = b+crop_height+gap
    
    page_type,intial_height = check_page_type(img,amend_page)
    
    if page_type == 1:
        intial_width = 193
        generate(intial_width,intial_width,intial_height,6)
        amend_page = False
    else:
        intial_width = 193
        generate(intial_width,intial_width,intial_height,40)
        amend_page = True
        
    return amend_page

def check_page_type(img,amend_page):
    
    return 1,550

    if amend_page:
        return 2,295
    
    a,b,c,d = 130, 280,800,155  # amend page check
    crop_img = crop_section(a,b,c,d,img)

    crop_temp_path = "temp.jpg"
    crop_img.save(crop_temp_path)
    
    text = (pytesseract.image_to_string(crop_temp_path, config='--psm 6', lang='eng+hin')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = text.split('\n')
    text = [ i for i in text if i!='' and i!='\x0c']
    
    if len(text)>0:
        for key in ['घटक','परिवर्धन']:
            for t in text:
                if key in t:
                    return 2,460                  
            
    return 1,270
    

def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def step2_order(params):
    for idx,value in enumerate(params):
        if difflib.SequenceMatcher(None,'निर्वाचक का नाम',value.split(":")[0]).ratio()>0.80:
            new_params = params[idx-1:]
            return new_params
    return params

def save_to_csv(dataframe, full_filepath):
    
    if dataframe.empty:
        return 
    else :       
        dataframe.to_csv(full_filepath,index=False)
        
def combine_all_csvs(combine_full_filepath,csvs_path):
    
    all_filenames = os.listdir(csvs_path)
    csvs_list = []
    
    for file in all_filenames:
        if file.endswith('csv'):
            csvs_list.append(pd.read_csv(csvs_path+file))
            
    combined_csv = pd.concat(csvs_list)
    combined_csv.to_csv( combine_full_filepath, encoding='utf-8-sig')


In [ ]:
def extract_4_numbers(crop_stat_path):
    
    text = (pytesseract.image_to_string(crop_stat_path, config='--psm 6', lang='eng+hin')) #config='--psm 4' config='-c preserve_interword_spaces=1'

    text = re.findall(r'\d+', text)    
    if len(text)==4:
        if int(text[0]) + int(text[1]) == int(text[2]):
            net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],"0",text[2]
        elif int(text[0]) + int(text[1]) == int(text[3]):
            net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],"0",text[3]
        else:
            net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],text[2],text[3]
    elif len(text) == 3 and int(text[2])>=int(text[1]) and int(text[2])>=int(text[0]):
        net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],"0",text[2]
    elif len(text) == 2 and int(text[0])*2-100<int(text[1]):
        net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],int(text[1])-int(text[0]),"0",text[1]
    else:
        net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = "","","",""
    
    return net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total

def fetch_last_page_content(path):
    img = Image.open(path)
    crop_path = input_images_blocks_path+"page/"
    create_path(crop_path)
    
    a,b,c,d = 2494, 990, 1200, 130 # last page 1st
    crop_img = crop_section(a,b,c,d,img)

    crop_last_path = crop_path+"last.jpg"
    crop_img.save(crop_last_path)
    
    a_1,b_1,c_1,d_1 = extract_4_numbers(crop_last_path)
    
    a,b,c,d = 2484, 1552, 1172, 120 # last page 2nd
    crop_img = crop_section(a,b,c,d,img)

    crop_last_path = crop_path+"last.jpg"
    crop_img.save(crop_last_path)
    
    a_2,b_2,c_2,d_2 = extract_4_numbers(crop_last_path)
    
    a,b,c,d = 2484, 1766, 1160, 80 # last page 3nd
    crop_img = crop_section(a,b,c,d,img)

    crop_last_path = crop_path+"last.jpg"
    crop_img.save(crop_last_path)
    
    a_3,b_3,c_3,d_3 = extract_4_numbers(crop_last_path)
    
    return a_1,b_1,c_1,d_1,a_2,b_2,c_2,d_2,a_3,b_3,c_3,d_3
    
    



In [ ]:
def arrane_columns(first_page_list,block_list,filename):
    
    year = 2017
    state = 'up'
    
    ac_name,parl_constituency,part_no,main_town,police_station,polling_station_name,polling_station_address,revenue_division,mandal,district,pin_code,net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = first_page_list
    
    name,rel_name,rel_type,house_no,age,gender,voter_id,number = block_list
    
    final_list = [number,voter_id,name,rel_name,rel_type,house_no,age,gender,ac_name,
                 parl_constituency,part_no,year,state,filename,main_town,police_station,mandal,
                 revenue_division,district,pin_code,polling_station_name,polling_station_address,
                 net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total]

    return final_list


In [ ]:
def split_data(data):
    seps = [":",">","-","."]
    
    for s in seps:
        if s in data:
            break

    data = data.split(s)
    data = [ i for i in data if i.strip()!='']
    if len(data)>1:
        data = data[1].strip()
        return data
    else:
        data = ""
        

In [ ]:
def extract_name(name):
    
    row = name.split(":")
    if len(row)!=2:
        return ""
    else:
        return row[1].strip()

In [ ]:
def extract_vid(v_id):
    row = v_id.split(" ")
    if len(row)==2:
        number = re.findall(r'\d+', row[0].strip())
        if len(number)>0:
            return number[0],row[1]
        else:
            
            return "",row[1]
    
    if len(row)==1:
        return "", row[-1]
    elif len(row)>2:
        number = re.findall(r'\d+', row[-2].strip())
        
        if len(number)>0:
            return number[0],row[-1]
        else:
            return "",row[-1]
    else:
        return "",""

In [ ]:
def extract_house_no(house_no):
    row = house_no.split(":")
    if len(row)==2:
        house_no = re.findall(r'\d+', row[1].strip())
        if len(house_no)>0:
            return house_no[0]
        else:
            return ""
    else:
        house_no = re.findall(r'\d+', row[0].strip())
        if len(house_no)>0:
            return house_no[0]
        else:
            return ""

In [ ]:
def extract_age_gender(age_gender):
    row = age_gender.split(":")
    
    if len(row)!=3:
        return "",""
    else:    
        age = re.findall(r'\d+', row[1].strip())
        if len(age)>0:
            age =  age[0]
        else:
            age = ""
            
        if 'महिला' in row[2].strip():
            gender = 'महिला'
            
        elif 'पुरूष' in row[2].strip():
            gender = 'पुरूष'
        else:
            gender =''

    return age, gender


In [ ]:
def extract_rel_name(rel_name):
    row = rel_name.split(":")
    if len(row)!=2:
        return "",""
    else:
        rel_type = extract_rel_type(row[0].strip())
        
        return row[1].strip(),rel_type
    
def extract_rel_type(rel_type):
    line = rel_type
    if line.startswith("पति") :
        rel_type = 'husband'
    elif line.startswith("पिता") :
        rel_type = 'father'
    elif line.startswith("माता") :
        rel_type = 'mother'
    elif line.startswith("अन्य") :
        rel_type = 'other'
    else:
        rel_type = ""
    
    return rel_type 


In [ ]:
def extract_details_from_block(block):
    
    v_id = block[0]
    name = block[1]
    rel_name = block[2]
    house_no = block[3]
    age_gender = block[4]
    
    name = extract_name(name)
    rel_name,rel_type = extract_rel_name(rel_name)
    house_no = extract_house_no(house_no)
    age, gender = extract_age_gender(age_gender)
    number,voter_id = extract_vid(v_id)
    
    return [name,rel_name,rel_type,house_no,age,gender,voter_id,number]


In [ ]:
def extract_first_page_details(path):
    
    img = Image.open(path)
    
    a,b,c,d = 1970,5264,1860,140  # stats for male and female
    crop_img = crop_section(a,b,c,d,img)

    crop_path = input_images_blocks_path+"page/"
    create_path(crop_path)
    
    crop_stat_path = crop_path+"stat.jpg"
    crop_img.save(crop_stat_path)

    a_n,b_n,c_n,d_n = extract_4_numbers(crop_stat_path)
    
    a,b,c,d = 2308,3029,1110,883  # mandal block
    crop_img = crop_section(a,b,c,d,img)
    
    crop_det_path = crop_path+"det.jpg"
    crop_img.save(crop_det_path)

    text = (pytesseract.image_to_string(crop_det_path, config='--psm 6', lang='eng+hin')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = text.split('\n')
    text = [ i for i in text if i!='' and i!='\x0c']
        
    if len(text) == 7:
        main_town,police_station,revenue_division,mandal,district,pin_code = split_data(text[0]),split_data(text[3]),split_data(text[2]),split_data(text[4]),split_data(text[5]),split_data(text[6])
    else:
        main_town,police_station,revenue_division,mandal,district,pin_code = extract_detail_section(text)
        
    
    a,b,c,d = 3190,490,750,155 # part no
    crop_img = crop_section(a,b,c,d,img)
    
    crop_part_path = crop_path+"part.jpg"
    crop_img.save(crop_part_path)

    text = (pytesseract.image_to_string(crop_part_path, config='--psm 6', lang='eng+hin')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = re.findall(r'\d+', text)
        
    if len(text)>0:
        part_no = text[0]
    else:
        part_no = ""
        
    
    a,b,c,d = 165,4105,1480,605 # police name name and address
    crop_img = crop_section(a,b,c,d,img)
    
    crop_police_path = crop_path+"police.jpg"
    crop_img.save(crop_police_path)

    text = (pytesseract.image_to_string(crop_police_path, config='--psm 6', lang='eng+hin')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = text.split('\n')
    text = [ i for i in text if i!='' and i!='\x0c']

    if len(text) == 4:
        polling_station_name, polling_station_address = text[1],text[3]
    else:
        polling_station_name, polling_station_address = text_police_compare(text)        
    
    a,b,c,d = 165,490,3606,365 # ac name and parl
    crop_img = crop_section(a,b,c,d,img)
    
    crop_ac_path = crop_path+"ac.jpg"
    crop_img.save(crop_ac_path)

    text = (pytesseract.image_to_string(crop_ac_path, config='--psm 6', lang='eng+hin')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = text.split('\n')
    text = [ i for i in text if i!='' and i!='\x0c']
    
    if len(text) == 2:
        ac_name = text[0].split("भाग")[0]
        ac_name,parl_constituency = split_data(ac_name),split_data(text[1])
    else:
        ac_name,parl_constituency = text_ac_parl_compare(text)
    
    return [ac_name,parl_constituency,part_no,main_town,police_station,polling_station_name,polling_station_address,revenue_division,mandal,district,pin_code,a_n,b_n,c_n,d_n]



In [ ]:
# for pdf_file_name in ['A163_S12A163P248.pdf']:
for pdf_file_name in state_pdfs_files:
    
    print(pdf_file_name)
    
    #create images,blocks and csvs paths for each file
    pdf_file_name_without_ext = pdf_file_name.split('.pdf')[0]
    input_pdf_images_path = PARSE_DATA_PAGES+pdf_file_name_without_ext+"/"
    create_path(input_pdf_images_path)

    input_images_blocks_path = PARSE_DATA_BLOCKS+pdf_file_name_without_ext+"/"
    create_path(input_images_blocks_path)
    
    #convert pdf into bunch of images
    pdf_2_images_list = pdf_to_img(state_pdfs_path+pdf_file_name, input_pdf_images_path,dpi=500)
     
    #sort pages for looping
    input_images = os.listdir(input_pdf_images_path)
    sort_nicely(input_images)
    
    #empty intial data
    df = pd.DataFrame(columns = COLUMNS)
    order_problem = []
    
    amend_page = False
#     extra_list = fetch_last_page_content(input_pdf_images_path+input_images[-1])
    
    #for each page, parse the data
    for page in input_images:
    
        page_full_path = input_pdf_images_path+page
        
        #extract first page content
        if page == '1.jpg':
            first_page_list = extract_first_page_details(page_full_path)
            continue

        #ingnore 2nd page and last page
        if page == '2.jpg' or input_images[-1] == page:
            continue
        
        if os.path.exists(PARSE_DATA_CSVS+pdf_file_name_without_ext+".csv"):
            print(pdf_file_name_without_ext+".csv", "already exists")
            break
            

        #loop from 3 page onwards
        if page.endswith('.jpg'):
            
            final_invidual_blocks = []
            blocks_path = input_images_blocks_path+"blocks/"
            create_path(blocks_path)

            page_idx = page.split(".jpg")[0] + "/"
            page_blocks_path = blocks_path+page_idx
            create_path(page_blocks_path)
                
            amend_page = generate_poll_blocks_from_page(page_full_path,page_blocks_path,amend_page)

            if amend_page:
                page_type = 'amendment'
            else:
                page_type = 'original'
            
            sorted_blocks = os.listdir(page_blocks_path)
            sort_nicely(sorted_blocks)
            
            for jpg_file in sorted_blocks:
                
                if jpg_file.endswith('.jpg') :

                    text = (pytesseract.image_to_string(page_blocks_path+jpg_file, config='--psm 6', lang='eng+hin')) #config='--psm 4' config='-c preserve_interword_spaces=1'
                    params_list = text.split('\n')
                    new_params_list = [ i for i in params_list if i!='' and i!='\x0c']
                    
                    if len(new_params_list) !=5:
                        order_problem.append((page, jpg_file,new_params_list))
                    else:
                        final_invidual_blocks.append(new_params_list)
       
        #put the data into dataframe
        for block in final_invidual_blocks:
            block_list = extract_details_from_block(block)
            final_list = arrane_columns(first_page_list,block_list,pdf_file_name_without_ext)
            final_list.append(page_type)
            
#             for i in extra_list:
#                 final_list.append(i)
                        
            df_length = len(df)
            df.loc[df_length] = final_list
        print("page done : ",page)
                
    #save the dataframe(pdf) data into csv
    save_to_csv(df,PARSE_DATA_CSVS+pdf_file_name_without_ext+".csv")
    print("CSV saved")
    print()


In [ ]:
#combine all state files into one csv
combine_all_csvs("final.csv",PARSE_DATA_CSVS)